In [1]:
import DataGenerator
import ShipDetection
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Input
from keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split

2023-01-29 00:56:37.941781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-29 00:56:39.098783: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-29 00:56:39.098978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-29 00:56:39.098987: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Set local variables

In [2]:
imgWidth = 256
imgHeight = 256
batch_size = 12
val_ratio = 0.2
trainDir = "data/airbus-ship-detection/train_v2/"

Set paths to data

In [3]:
train_ship_segmentations_v2 = pd.read_csv("data/airbus-ship-detection/train_ship_segmentations_v2.csv")
train_ship_segmentations_v2.set_index("ImageId", inplace=True)
withShipsDf = train_ship_segmentations_v2[train_ship_segmentations_v2["EncodedPixels"].isna()==False]
paths = trainDir+withShipsDf.index.to_numpy()[int(len(withShipsDf)/2):]
val_amount = int(len(paths)*val_ratio)
trainPaths, validationPaths = train_test_split(paths, test_size=val_amount)

Create Data generators for model

In [4]:
training_generator = DataGenerator.DataGenerator(trainPaths,
                                                 train_ship_segmentations_v2,
                                                 batch_size=batch_size)
validation_generator = DataGenerator.DataGenerator(validationPaths,
                                                   train_ship_segmentations_v2,
                                                   batch_size=batch_size)

Create model

In [57]:
inputImg = Input((imgWidth, imgHeight, 3), name='img')
model = ShipDetection.get_unet(inputImg)
model.compile(optimizer=Adam(learning_rate=0.01), loss=ShipDetection.jacard_coef_loss,metrics=[ShipDetection.jacard_coef], run_eagerly=True)

In [58]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 img (InputLayer)               [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_96 (Conv2D)             (None, 256, 256, 16  448         ['img[0][0]']                    
                                )                                                                 
                                                                                                  
 batch_normalization_91 (BatchN  (None, 256, 256, 16  64         ['conv2d_96[0][0]']              
 ormalization)                  )                                                           

Set fitting callbacks

In [59]:
callbacks = [
    EarlyStopping(patience=2, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=1, min_lr=0.00001, verbose=1)
]

Fit model

In [61]:
results = model.fit(training_generator,
                    validation_data=validation_generator,
                    epochs=20,
                    callbacks=callbacks,
                    verbose=1
                    )

Epoch 1/20
2724/2724 [==============================] - 973s 357ms/step - loss: -0.4360 - jacard_coef: 0.4360 - val_loss: -0.4693 - val_jacard_coef: 0.4693 - lr: 0.0100
Epoch 2/20
2724/2724 [==============================] - 970s 356ms/step - loss: -0.5197 - jacard_coef: 0.5197 - val_loss: -0.5304 - val_jacard_coef: 0.5304 - lr: 0.0100
Epoch 3/20
2724/2724 [==============================] - 969s 356ms/step - loss: -0.5575 - jacard_coef: 0.5575 - val_loss: -0.5547 - val_jacard_coef: 0.5547 - lr: 0.0100
Epoch 4/20
2724/2724 [==============================] - ETA: 0s - loss: -0.5847 - jacard_coef: 0.5847
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
2724/2724 [==============================] - 966s 355ms/step - loss: -0.5847 - jacard_coef: 0.5847 - val_loss: -0.5539 - val_jacard_coef: 0.5539 - lr: 0.0100
Epoch 5/20
2724/2724 [==============================] - 969s 356ms/step - loss: -0.6311 - jacard_coef: 0.6311 - val_loss: -0.6418 - val_jacard_coef: 0.6418 -

In [62]:
model.save("model_jaccard")

INFO:tensorflow:Assets written to: model_jaccard/assets


INFO:tensorflow:Assets written to: model_jaccard/assets
